In [1]:
import pandas as pd
import numpy as np
import os

In [229]:
#this combines the files together in the directories
def combine_files_together():
    base_path = "data/"
    elms_path = "ELMS"
    lmc_cup_path = "LeMansCup"
    fiawec_path = "FIA WEC"
    imsa_path = "IMSA"

    file_paths = [elms_path, lmc_cup_path, fiawec_path, imsa_path]
    #combine the dfs together
    for f in file_paths:
        curr_dir = base_path + f + " Data/"
        full_df = pd.DataFrame()
        current_circuit = ""
        round = 1
        for filename in os.listdir(curr_dir):
            file = os.path.join(curr_dir, filename)
            df = pd.read_csv(file, index_col=0)
            full_df = full_df.append(df)
        full_csv_loc = f + "_full_data.csv"
        full_df = full_df.dropna(axis=1, how = "all")
        full_df = full_df.reset_index(drop=True)
        full_df.columns = full_df.columns.str.strip()
        full_df.columns = full_df.columns.str.lower()
        full_df.to_csv(full_csv_loc)

In [3]:
#load all the combined dfs
def load_all_dfs():
    fia_wec_df = pd.read_csv('FIA WEC_full_data_original.csv', index_col = 0)
    imsa_df = pd.read_csv('IMSA_full_data_original.csv', index_col = 0)
    elms_df = pd.read_csv('ELMS_full_data_original.csv', index_col = 0)
    lmc_df = pd.read_csv('LeMansCup_full_data_original.csv', index_col = 0)
    full_df = fia_wec_df.append(imsa_df).append(elms_df).append(lmc_df)
    full_df = full_df.reset_index(drop=True)

In [231]:
#this method fixes the round
def fix_round():
    full_df = full_df.sort_values(['championship', 'season', 'round']).reset_index(drop=True)
    #group by the champ, season, and round to fix the round
    groupby_df = full_df.groupby(['championship', 'season', 'round']).mean().reset_index()
    groupby_df['new_round'] = 1

    #use cumsum on the new round based on championship and season 
    groupby_df['new_round'] = groupby_df.groupby(['championship', 'season'])['new_round'].cumsum()

    #create a key based off the champ, season, and previous round
    groupby_df['key'] = groupby_df['championship'] + "_" + groupby_df['season'].map(str) + "_" + groupby_df['round'].map(int).map(str)
    wanted_columns = ['key', 'new_round']
    groupby_df = groupby_df[wanted_columns] #minmize the df to the key + the fixed round

    #put the same key on the full_df
    full_df['key'] = full_df['championship'] + "_" + full_df['season'].map(str) + "_" + full_df['round'].map(int).map(str)
    #and then merge the dfs together on the key
    full_df_round_fix = pd.merge(full_df, groupby_df, on = 'key')

    #drop the new round and key 
    full_df_round_fix['round'] = full_df_round_fix['new_round']
    full_df_round_fix = full_df_round_fix.drop(['new_round', 'key'], axis=1)

In [2]:
#sort the df by championship, season and round, the round should be correct but not in the right number order
def load_df():
    full_df = pd.read_csv("alkamel_systems_full_data.csv", index_col=0)
    return full_df
full_df = load_df()

C:\Users\trist\AppData\Local\Temp\ipykernel_25428\4247207565.py:5: DtypeWarning: Columns (21,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df = load_df()


In [233]:
#okay so the next thing to prob fix to drop porsche,ford,t6,t7, not used really for anything
#also fix s1,s2,s3, pit_time and convert to seconds
full_df.columns

Index(['number', 'driver_number', 'lap_number', 'lap_time', 'lap_improvement',
       'crossing_finish_line_in_pit', 's1', 's1_improvement', 's2',
       's2_improvement', 's3', 's3_improvement', 'kph', 'elapsed', 'hour',
       'top_speed', 'driver_name', 'pit_time', 'championship', 'session_type',
       'season', 'circuit', 'round', 'class', 'group', 'team', 'manufacturer',
       'flag_at_fl', 'lap_time_seconds', 's1_seconds', 's2_seconds',
       's3_seconds', 'elapsed_seconds', 'pit_time_seconds', 'session'],
      dtype='object')

In [234]:
#drop the ones we don't need like the le mans specific ones
full_df_without_lm = full_df.drop(['porsche', 'ford', 't6', 't7'], axis=1)

#here we're going to fix the sectors, this is just showing why we would need to fix those
wanted_columns = ['s1', 's2', 's3', 'pit_time', 'elapsed',  's1_seconds', 's2_seconds', 's3_seconds', 's1_large', 's2_large', 's3_large']
full_df_sectors = full_df_without_lm[wanted_columns]
full_df_sectors.head(60)

KeyError: "['porsche' 'ford' 't6' 't7'] not found in axis"

In [ ]:
full_df_without_lm = full_df_without_lm.drop(['s1_seconds', 's2_seconds', 's3_seconds', 's1_large', 's2_large', 's3_large'], axis=1)
full_df_without_lm.head(60)

In [ ]:
#this we use to convert anything to ms
def convert_to_s(x):
    if(x == np.nan):
        return x
    hour = 0
    minutes = 0
    try:
        hour, minutes, second = x.split(":")
    except ValueError:
        hour = 0
        try:
            minutes, second = x.split(":")
        except ValueError:
            hour = 0
            minutes = 0
            second = x
    except AttributeError:
        print(x)
    hour_seconds = int(hour)*60*60
    if hour == 0 and minutes == 0:
        return x
    else:
        ms = 0
        try:
            seconds, ms = second.split(".")
        except ValueError:
            seconds = second
    minute_seconds = (int(minutes)*60 + int(seconds) + int(hour_seconds))*1000 
    total_ms = minute_seconds + int(ms)
    return total_ms/1000

#this uses the method to convert all of the large ones (h:m:s.ms to seconds)
def convert_df_cols_to_seconds():
    full_df_without_lm['lap_time_seconds'] = full_df_without_lm.lap_time.map(convert_to_s)
    full_df_without_lm['s1_seconds'] = full_df_without_lm.s1.map(convert_to_s)
    full_df_without_lm['s2_seconds'] = full_df_without_lm.s2.map(convert_to_s)
    full_df_without_lm['s3_seconds'] = full_df_without_lm.s3.map(convert_to_s)
    full_df_without_lm['elapsed_seconds'] = full_df_without_lm.elapsed.map(convert_to_s)
    full_df_without_lm['pit_time_seconds'] = full_df_without_lm.pit_time.map(convert_to_s)
convert_df_cols_to_seconds()

In [ ]:
full_df_without_lm['session'] = full_df_without_lm['session_type']
def map_sessions(s):
    if ('QUALIFYING' in s):
        return 'QUALIFYING'
    elif('SESSION' in s or 'TEST' in s):
        return 'TEST'
    elif ('PRACTICE' in s):
        return 'FREE PRACTICE'
    elif ('RACE' in s):
        return 'RACE'
    else:
        return s
#maps the sessions to session_types
full_df_without_lm['session_type'] = full_df_without_lm['session'].map(map_sessions)
full_df_without_lm_test = full_df_without_lm.groupby(['session','session_type']).mean()
full_df_without_lm_test

,session_type,session,number,driver_number,lap_number,lap_improvement,s1_improvement,s2_improvement,s3_improvement,kph,top_speed,round,pit_time_seconds
0,FREE PRACTICE,FREE PRACTICE,49.168802,1.149631,17.359862,0.065463,0.124567,0.124942,0.125035,174.477640,286.864056,4.061162,768.597400
1,FREE PRACTICE,FREE PRACTICE 1,46.363661,1.274209,16.988555,0.068511,0.117419,0.118022,0.118360,153.839584,253.728908,4.524759,267.169055
2,FREE PRACTICE,FREE PRACTICE 2,46.044549,1.261306,17.846565,0.065773,0.113201,0.114025,0.114057,156.666782,255.102336,4.498956,229.513011
3,FREE PRACTICE,FREE PRACTICE 3,55.890584,1.306482,14.176051,0.079351,0.140683,0.141097,0.141432,158.287327,259.901657,4.779552,204.229663
4,FREE PRACTICE,FREE PRACTICE 4,61.698340,1.827088,11.133565,0.107602,0.185493,0.185760,0.184957,187.258753,288.618864,3.728319,272.988220
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,TEST,TUESDAY MORNING SESSION,55.750000,0.803571,25.975174,0.043118,0.069686,0.070122,0.071429,147.995862,269.946474,1.000000,861.304642
88,TEST,TUESDAY NIGHT SESSION,51.369128,1.037584,16.186577,0.080537,0.126174,0.123490,0.124832,155.173960,271.415973,1.000000,682.354042
89,TEST,WEDNESDAY AFTERNOON SESSION,56.749580,0.953501,24.454342,0.049300,0.078992,0.076751,0.078431,153.695742,264.824986,1.000000,694.180439
90,TEST,WEDNESDAY MORNING SESSION,56.073968,0.940395,32.473968,0.034829,0.059605,0.059246,0.059605,153.046140,267.185889,1.000000,672.478587


In [74]:
full_df = load_df()

C:\Users\trist\AppData\Local\Temp\ipykernel_13508\207732749.py:1: DtypeWarning: Columns (21,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df = load_df()


In [8]:
full_df[full_df['team'].isna()].groupby(['championship', 'season']).mean()

number  driver_number  lap_number  lap_improvement  \
championship season                                                          
ELMS         2013    48.682078       0.691054   32.698913         0.055259   
             2014    62.895294       0.924538   38.618216         0.046998   
FIA WEC      2012    39.577170       0.900538   68.335338         0.042167   
             2013    50.891440       0.926733   71.421456         0.039764   
             2014    49.548533       0.940917   80.853928         0.036858   

                     s1_improvement  s2_improvement  s3_improvement  \
championship season                                                   
ELMS         2013          0.079468        0.081360        0.082525   
             2014          0.075866        0.076184        0.076819   
FIA WEC      2012          0.066227        0.066439        0.066348   
             2013          0.066883        0.067557        0.067489   
             2014          0.063481        0.063678        0.064014   

                            kph   top_speed     round  lap_time_seconds  \
championship season                                                       
ELMS         2013    152.981273  246.870534  3.782505        145.056429   
             2014    153.117916  256.968944  3.757766        156.993400   
FIA WEC      2012    161.940663  249.674260  4.407747        157.737871   
             2013    164.636609  255.484643  5.206208        197.863855   
             2014    174.092409  268.903853  3.825176        211.356116   

                     s1_seconds  s2_seconds  s3_seconds  elapsed_seconds  \
championship season                                                        
ELMS         2013     52.387327   43.720303   46.148984      4214.822334   
             2014     61.839816   41.759286   51.448264      5338.941614   
FIA WEC      2012     62.825263   49.644845   44.252677      8988.228104   
             2013     72.782479   62.379440   62.518420     14220.282605   
             2014     74.464243   62.943784   73.926810     17185.022917   

                     pit_time_seconds  
championship season                    
ELMS         2013          301.686146  
             2014          406.396495  
FIA WEC      2012          320.632637  
             2013          336.219259  
             2014          397.776590

In [65]:
#some nas here
full_df[~full_df['team'].isna()]

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,team,manufacturer,flag_at_fl,lap_time_seconds,s1_seconds,s2_seconds,s3_seconds,elapsed_seconds,pit_time_seconds,session
55252,2,1,1,28:16.841,0,NaN,26:46.611,0,32.847,0,...,Team LNT,Nissan,NaN,1696.841,1606.611,32.847,57.383,1696.841,1565.947,AFTERNOON SESSION DAY 1
55253,2,1,2,2:06.609,0,NaN,38.049,0,31.993,0,...,Team LNT,Nissan,NaN,126.609,38.049,31.993,56.567,1823.450,NaN,AFTERNOON SESSION DAY 1
55254,2,1,3,2:04.283,0,NaN,37.492,0,31.533,0,...,Team LNT,Nissan,NaN,124.283,37.492,31.533,55.258,1947.733,NaN,AFTERNOON SESSION DAY 1
55255,2,1,4,2:04.528,0,NaN,37.322,1,31.660,0,...,Team LNT,Nissan,NaN,124.528,37.322,31.660,55.546,2072.261,NaN,AFTERNOON SESSION DAY 1
55256,2,1,5,2:16.340,0,B,37.334,0,31.119,1,...,Team LNT,Nissan,NaN,136.340,37.334,31.119,67.887,2208.601,NaN,AFTERNOON SESSION DAY 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039667,99,2,43,1:51.074,0,NaN,34.909,0,37.686,0,...,Bullitt Racing,Aston Martin,GF,111.074,34.909,37.686,38.479,6149.383,NaN,RACE
2039668,99,2,44,2:37.120,0,NaN,35.372,0,37.713,0,...,Bullitt Racing,Aston Martin,FCY,157.120,35.372,37.713,84.035,6306.503,NaN,RACE
2039669,99,2,45,3:23.332,0,NaN,1:24.748,0,1:19.029,0,...,Bullitt Racing,Aston Martin,GF,203.332,84.748,79.029,39.555,6509.835,NaN,RACE
2039670,99,2,46,1:51.094,0,NaN,35.335,0,37.742,0,...,Bullitt Racing,Aston Martin,GF,111.094,35.335,37.742,38.017,6620.929,NaN,RACE


In [91]:
#here we have to download the classifications, here I basically loaded the classifications and pulled all the ones that we have to match up

wanted_columns = ['key', 'team','class']
def fix_fiawec_nas():
    classification_2012 = pd.read_csv("2012_FIAWEC_classification.csv", sep=";")
    classification_2013 = pd.read_csv("2013_FIAWEC_classification.csv", sep=";")
    classification_2014 = pd.read_csv("2014_FIAWEC_classification.csv", sep=";")
    classification_2012['key'] = "2012_" + "FIA WEC_" +  classification_2012['NUMBER'].map(str)
    classification_2013['key'] = "2013_" + "FIA WEC_" +  classification_2013['NUMBER'].map(str)
    classification_2014['key'] = "2014_" + "FIA WEC_" +  classification_2014['NUMBER'].map(str)

    fiawec_classifications = classification_2014.append(classification_2013).append(classification_2012)
    fiawec_classifications.columns = fiawec_classifications.columns.str.lower()
    fiawec_classifications = fiawec_classifications[wanted_columns]
    fiawec_r8_df = pd.read_csv('2013_FIAWEC_8_nas.csv', sep = ';')
    fiawec_r8_df['key'] = "2013_FIA WEC_" +  fiawec_r8_df['NUMBER'].map(str)
    fiawec_r8_df.columns = fiawec_r8_df.columns.str.lower()
    fiawec_r8_df = fiawec_r8_df[wanted_columns]

    fiawec_r4_2012_df = pd.read_csv('2012_r4_classification.csv', sep = ';')
    fiawec_r4_2012_df['key'] = "2012_FIA WEC_" + fiawec_r4_2012_df['NUMBER'].map(str)
    fiawec_r4_2012_df.columns = fiawec_r4_2012_df.columns.str.lower()
    fiawec_r4_2012_df = fiawec_r4_2012_df[wanted_columns]

    fiawec_test_df = pd.read_csv('2012_LM_test_classification.csv', sep = ';')
    fiawec_test_df['key'] = "2012_FIA WEC_" +  fiawec_test_df['NUMBER'].map(str)
    fiawec_test_df.columns = fiawec_test_df.columns.str.lower()
    fiawec_test_df = fiawec_test_df[wanted_columns]
    
    fiawec_test_df = pd.read_csv('2014_test_classification.csv', sep = ';')
    fiawec_test_df['key'] = "2014_FIA WEC_" +  fiawec_test_df['NUMBER'].map(str)
    fiawec_test_df.columns = fiawec_test_df.columns.str.lower()
    fiawec_test_df = fiawec_test_df[wanted_columns]

    fiawec_r5_df = pd.read_csv('2014_r5_classification.csv', sep = ';')
    fiawec_r5_df['key'] = "2014_FIA WEC_" +  fiawec_r5_df['NUMBER'].map(str)
    fiawec_r5_df.columns = fiawec_r5_df.columns.str.lower()
    fiawec_r5_df = fiawec_r5_df[wanted_columns]

    fiawec_2012_test_classification = pd.read_csv('2012_fiawec_test_classification.csv', sep = ';')
    fiawec_2012_test_classification['key'] = "2012_FIA WEC_" + fiawec_2012_test_classification['NUMBER'].map(str)
    fiawec_2012_test_classification.columns = fiawec_2012_test_classification.columns.str.strip().str.lower()
    fiawec_2012_test_classification = fiawec_2012_test_classification[wanted_columns]

    fiawec_2014_test_classification = pd.read_csv('2014_fiawec_test_classification.csv', sep = ';')
    fiawec_2014_test_classification['key'] = "2014_FIA WEC_" + fiawec_2014_test_classification['NUMBER'].map(str)
    fiawec_2014_test_classification.columns = fiawec_2014_test_classification.columns.str.strip().str.lower()
    fiawec_2014_test_classification = fiawec_2014_test_classification[wanted_columns]

    fiawec_classifications_with_nas = fiawec_classifications.append(fiawec_r5_df).append(fiawec_test_df).append(fiawec_r8_df).append(fiawec_test_df).append(fiawec_r4_2012_df)\
        .append(fiawec_2014_test_classification).append(fiawec_2012_test_classification)
    fiawec_classifications_with_nas = fiawec_classifications_with_nas.groupby(['key', 'team','class']).mean().reset_index()
    return fiawec_classifications_with_nas
def fix_missing_elms_classification():
    elms_2013_classification = pd.read_csv('2013_elms_classification.csv', sep = ';')
    elms_2013_classification['key'] = "2013_ELMS_" + elms_2013_classification['NUMBER'].map(str)
    elms_2013_classification.columns = elms_2013_classification.columns.str.strip().str.lower()
    elms_2013_classification = elms_2013_classification[wanted_columns]

    elms_2013_test_classification = pd.read_csv('2013_test_classification.csv', sep = ';')
    elms_2013_test_classification['key'] = "2013_ELMS_" + elms_2013_test_classification['NUMBER'].map(str)
    elms_2013_test_classification.columns = elms_2013_test_classification.columns.str.strip().str.lower()
    elms_2013_test_classification = elms_2013_test_classification[wanted_columns]

    elms_2014_classification = pd.read_csv('2014_elms_classification.csv', sep = ';')
    elms_2014_classification['key'] = "2014_ELMS_" +  elms_2014_classification['NUMBER'].map(str)
    elms_2014_classification.columns = elms_2014_classification.columns.str.strip().str.lower()
    elms_2014_classification = elms_2014_classification[wanted_columns]

    elms_2014_test_classification = pd.read_csv('2014_test_classification.csv', sep = ';')
    elms_2014_test_classification['key'] = "2014_ELMS_" + elms_2014_test_classification['NUMBER'].map(str)
    elms_2014_test_classification.columns = elms_2014_test_classification.columns.str.strip().str.lower()
    elms_2014_test_classification = elms_2014_test_classification[wanted_columns]

    elms_classification = elms_2013_classification.append(elms_2013_test_classification).append(elms_2014_classification).append(elms_2014_test_classification)
    return elms_classification
fiawec_classifications_with_nas = fix_fiawec_nas()
elms_classification_with_nas = fix_missing_elms_classification()
classifications = fiawec_classifications_with_nas.append(elms_classification_with_nas)
classifications = classifications.groupby(['key','team','class']).mean().reset_index().sort_values('key').to_csv('classifications_with_class.csv')
classifications

In [51]:
def load_df():
    full_df = pd.read_csv("alkamel_systems_full_data.csv", index_col=0)
    return full_df
full_df = load_df()

C:\Users\trist\AppData\Local\Temp\ipykernel_25428\2373733509.py:4: DtypeWarning: Columns (21,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df = load_df()


In [101]:
classifications_with_class = pd.read_csv('classifications_with_class.csv',index_col=0)
classifications_with_class

,key,team,class
0,2012_FIA WEC_0,Highcroft Racing,CDNT
1,2012_FIA WEC_1,Audi Sport Team Joest,LMP1
2,2012_FIA WEC_12,Rebellion Racing,LMP1
3,2012_FIA WEC_13,Rebellion Racing,LMP1
4,2012_FIA WEC_15,Oak Racing,LMP1
...,...,...,...
260,2014_FIA WEC_92,Porsche Team Manthey,LMGTE Pro
261,2014_FIA WEC_95,Aston Martin Racing,LMGTE Am
262,2014_FIA WEC_97,Aston Martin Racing,LMGTE Pro
263,2014_FIA WEC_98,Aston Martin Racing,LMGTE Am


In [108]:
fiawec_nas = full_df[(full_df['team'].isna())&(full_df['championship']=='FIA WEC')]
elms_nas = full_df[(full_df['team'].isna())&(full_df['championship']=='ELMS')]

fiawec_nas['key'] = fiawec_nas['season'].map(str) + "_FIA WEC_" + fiawec_nas['number'].map(str)
elms_nas['key'] = elms_nas['season'].map(str) + "_ELMS_" + elms_nas['number'].map(str)

combined_nas = fiawec_nas.append(elms_nas)
na_fix = pd.merge(combined_nas, classifications_with_class, on = 'key', how = 'left')
na_fix['team'] = na_fix['team_y']
na_fix['class'] = na_fix['class_y']
na_fix = na_fix.drop(['key','team_y','team_x', 'class_x', 'class_y'],axis=1)

team_not_nas = full_df[~full_df['team'].isna()]
full_df_with_fix = team_not_nas.append(na_fix)
len(full_df_with_fix) - len(full_df)

C:\Users\trist\AppData\Local\Temp\ipykernel_25428\3781170966.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fiawec_nas['key'] = fiawec_nas['season'].map(str) + "_FIA WEC_" + fiawec_nas['number'].map(str)
C:\Users\trist\AppData\Local\Temp\ipykernel_25428\3781170966.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elms_nas['key'] = elms_nas['season'].map(str) + "_ELMS_" + elms_nas['number'].map(str)


0

In [109]:
full_df_with_fix.to_csv('alkamel_systems_full_data_with_teams.csv')

In [116]:
full_df_with_fix = full_df_with_fix.sort_values(['championship', 'season','round']).reset_index(drop=True)
full_df_with_fix.to_csv('alkamel_systems_full_data_with_teams.csv')


,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,team,manufacturer,flag_at_fl,lap_time_seconds,s1_seconds,s2_seconds,s3_seconds,elapsed_seconds,pit_time_seconds,session
0,1,1,1,1:20:16.192,0,NaN,1:18:52.104,0,33.658,0,...,Thiriet by TDS Racing,NaN,NaN,4816.192,4732.104,33.658,50.430,4816.192,4696.0,TUESDAY AFTERNOON SESSION
1,1,1,2,1:51.869,0,NaN,34.073,0,29.034,0,...,Thiriet by TDS Racing,NaN,NaN,111.869,34.073,29.034,48.762,4928.061,NaN,TUESDAY AFTERNOON SESSION
2,1,1,3,1:51.577,0,NaN,34.017,0,29.004,0,...,Thiriet by TDS Racing,NaN,NaN,111.577,34.017,29.004,48.556,5039.638,NaN,TUESDAY AFTERNOON SESSION
3,1,1,4,1:51.662,0,NaN,33.867,0,28.969,0,...,Thiriet by TDS Racing,NaN,NaN,111.662,33.867,28.969,48.826,5151.300,NaN,TUESDAY AFTERNOON SESSION
4,1,1,5,1:50.943,0,NaN,33.948,0,28.768,0,...,Thiriet by TDS Racing,NaN,NaN,110.943,33.948,28.768,48.227,5262.243,NaN,TUESDAY AFTERNOON SESSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039667,99,2,43,1:51.074,0,NaN,34.909,0,37.686,0,...,Bullitt Racing,Aston Martin,GF,111.074,34.909,37.686,38.479,6149.383,NaN,RACE
2039668,99,2,44,2:37.120,0,NaN,35.372,0,37.713,0,...,Bullitt Racing,Aston Martin,FCY,157.120,35.372,37.713,84.035,6306.503,NaN,RACE
2039669,99,2,45,3:23.332,0,NaN,1:24.748,0,1:19.029,0,...,Bullitt Racing,Aston Martin,GF,203.332,84.748,79.029,39.555,6509.835,NaN,RACE
2039670,99,2,46,1:51.094,0,NaN,35.335,0,37.742,0,...,Bullitt Racing,Aston Martin,GF,111.094,35.335,37.742,38.017,6620.929,NaN,RACE


In [118]:
full_df_with_fix[full_df_with_fix['team'].isna()]

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,team,manufacturer,flag_at_fl,lap_time_seconds,s1_seconds,s2_seconds,s3_seconds,elapsed_seconds,pit_time_seconds,session
